# Validate Born Raytracing Against Theory

This notebook processes a Gower Street N-body simulation through a Born-approximation
raytracing pipeline with DES Y3 n(z), and validates the output convergence power spectra
against CAMB theory predictions.

**Prerequisites:** Run `bash scripts/download_data.sh` to download sim00001 and the DES Y3 FITS file.

In [ ]:
import sys
sys.path.insert(0, "..")

import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

from lensing.io import load_sim_params, load_shell_info, load_des_y3_nz, get_valid_shell_ids
from lensing.cosmology import setup_camb, get_theory_cls
from lensing.raytracing import compute_lensing_weights, born_raytrace
from lensing.validation import measure_all_cls, plot_validation, plot_ratio, plot_convergence_maps

%matplotlib inline
plt.rcParams['figure.dpi'] = 120

## 1. Load simulation parameters

In [ ]:
cosmo = load_sim_params("../gower_street_runs.csv", sim_id=1)
print("Cosmological parameters for sim00001:")
for k, v in cosmo.items():
    print(f"  {k:12s} = {v}")

## 2. Load shell info and DES Y3 n(z)

In [ ]:
SIM_DIR = "../data/sim00001"
FITS_PATH = "../data/2pt_NG_final_2ptunblind_02_26_21_wnz_maglim_covupdate.fits"

shell_info = load_shell_info(SIM_DIR)
print(f"Number of shells: {len(shell_info)}")
print(f"Redshift range: {shell_info['z_near'].min():.3f} to {shell_info['z_far'].max():.3f}")
print(f"Chi range (Mpc/h): {shell_info['chi_near'].min():.1f} to {shell_info['chi_far'].max():.1f}")

valid_ids = get_valid_shell_ids(SIM_DIR)
print(f"Valid shell count: {len(valid_ids)}")

In [ ]:
z_nz, nz_bins = load_des_y3_nz(FITS_PATH)

fig, ax = plt.subplots(figsize=(8, 4))
for i in range(4):
    ax.plot(z_nz, nz_bins[i], label=f"Bin {i+1}")
ax.set_xlabel("z")
ax.set_ylabel("n(z) [normalised]")
ax.legend()
ax.set_title("DES Y3 MagLim source n(z)")
plt.show()

## 3. Compute lensing weights

In [ ]:
weights = compute_lensing_weights(shell_info, z_nz, nz_bins, cosmo)
print(f"Weight array shape: {weights.shape}  (n_bins x n_shells)")

fig, ax = plt.subplots(figsize=(8, 4))
for i in range(4):
    ax.plot(shell_info['z_mid'], weights[i], label=f"Bin {i+1}")
ax.set_xlabel("Shell midpoint redshift")
ax.set_ylabel("Lensing weight")
ax.legend()
ax.set_title("Born lensing weights per shell")
plt.show()

## 4. Born raytracing

In [ ]:
NSIDE_OUT = 512

print(f"Running Born raytracing (nside_out={NSIDE_OUT})...")
kappa_maps = born_raytrace(SIM_DIR, shell_info, weights, nside_out=NSIDE_OUT)
print("Done.")

for i, km in enumerate(kappa_maps):
    print(f"  Bin {i+1}: mean={km.mean():.2e}, std={km.std():.4f}, min={km.min():.4f}, max={km.max():.4f}")

## 5. Convergence maps

In [ ]:
for i in range(4):
    hp.mollview(kappa_maps[i], title=f"Convergence — Bin {i+1}", min=-0.03, max=0.03)
    plt.show()

## 6. CAMB theory power spectra

In [ ]:
LMAX = 3 * NSIDE_OUT - 1

print(f"Computing CAMB theory C_ell up to lmax={LMAX}...")
results = setup_camb(cosmo, z_nz, nz_bins, lmax=LMAX)
theory_cls = get_theory_cls(results, lmax=LMAX)
print(f"Got {len(theory_cls)} spectra.")

## 7. Measure power spectra from maps

In [ ]:
measured_cls = measure_all_cls(kappa_maps, lmax=LMAX)
print(f"Measured {len(measured_cls)} spectra up to lmax={LMAX}.")

## 8. Validation: Measured vs Theory

In [ ]:
fig = plot_validation(measured_cls, theory_cls, lmax=LMAX)
plt.show()

In [ ]:
fig = plot_ratio(measured_cls, theory_cls, lmax=LMAX)
plt.show()

## 9. Discussion

**Expected behavior:**
- At low ell (< ~100): agreement within cosmic variance. For a single full-sky realization, the variance is $\sqrt{2/(2\ell+1)} \cdot C_\ell$.
- At intermediate ell (~100–500): should see good agreement (ratio close to 1.0).
- At high ell (> ~500): may see deviations due to:
  - Non-linear physics beyond Halofit (baryonic effects, higher-order perturbation theory)
  - N-body shot noise from finite particle number
  - Finite box effects (missing large-scale modes)
  - Born approximation vs full raytracing differences

These deviations are expected and informative — they quantify the limitations of the Born approximation and the Halofit non-linear prescription.